In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "30"

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.rcParams['font.size'] = 15

from utils.utils import *

import utils.HTC_utils as HTC
import random

import networkx as nx

import os
import seaborn as sns

cs = ['cornflowerblue', '#FEBE00']

In [ ]:
def plot_results(res, res2=None, cs=['cornflowerblue', 'orange'], labels=['control', 'stroke'], lw=3):
    plt.figure(figsize=(10,3))

    ax1 = plt.subplot(1,2,1)
    ax2 = ax1.twinx()

    ax1.plot(res[0], res[1], c=cs[0], label=labels[0], lw=lw)
    ax2.plot(res[0], res[2], c=cs[0], ls='-.', lw=lw)
    
    if res2 is not None:
        ax1.plot(res2[0], res2[1], c=cs[1], label=labels[1], lw=lw)
        ax2.plot(res2[0], res2[2], c=cs[1], ls='-.', lw=lw)

    ax1.set_xlabel('T')
    ax1.set_ylabel('A')
    ax2.set_ylabel(r'$\sigma$(A)')

    ax1 = plt.subplot(1,2,2)
    ax2 = ax1.twinx()

    ax1.plot(res[0], res[3], c=cs[0], label=labels[0], lw=lw)
    ax2.plot(res[0], res[4], c=cs[0], ls='-.', lw=lw)
    
    if res2 is not None:
        ax1.plot(res2[0], res2[3], c=cs[1], label=labels[1], lw=lw)
        ax2.plot(res2[0], res2[4], c=cs[1], ls='-.', lw=lw)

    ax1.set_xlabel('T') 
    ax1.set_ylabel('S1')
    ax2.set_ylabel('S2')
    
    ax1.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
### Load data
ses = 1
N = 500

mats = []

data = data_loader(which='control', ses=ses, parc=N, thr='mask', include_subctx=False)
W = data.load_matrix(13)
mats.append(W)

data = data_loader(which='stroke', ses=ses, parc=N, thr='mask', include_subctx=False)
W = data.load_matrix(13)
mats.append(W)

In [ ]:
### Define HTC parameters
# Rodrigo -> r1 = 2/ N; r2 = r2 ** (1/5)
# N=100 -> r1 = 1/N; r2 = 5e-1
# N=200 -> r1 = 1/N; r2 = 3e-1
# N=500 -> r1 = 1/N; r2 = 0.3
# N = 500

r1 = 1/N
r2 = 125/N

Tmin, Tmax = HTC.get_Trange(r1, r2)

steps = int(1e4)
eq_steps = int(5e3)
runs = 30
nT = 30 #nT = 30

# Run all dataset

In [ ]:
folder = 'results/criticality/'

tmp_dicts = [dict_control, dict_stroke]

steps = int(1e4)
eq_steps = int(5e3)
runs = 30
nT = 30

r2s = {100: 0.5, 200: 0.3, 500: 0.3}

for tmp_dict in tmp_dicts:
    which = tmp_dict['name']
    
    for parc in tmp_dict['parcs'][:-1]:
        for ses in tmp_dict['sessions'][:-1]:
            # Load dataset
            data = data_loader(which=which, ses=ses, parc=parc, thr='mask', include_subctx=False)
            
            for idx, mat in enumerate(data):
                # Check if connected - otherwise it has returned None
                if mat is None:
                    continue
                
                ### Define params
                r1 = 1/parc
                r2 = r2s[parc]

                Tminus = r1 * r2 / (r1 + r2 + r1*r2)
                Tplus = r2 / (2*r2 +1)

                xplus = Tplus
                yplus = Tplus / r2

                xminus = Tminus
                yminus = Tminus / r2

                Tmin = 0
                Tmax = 1.4 * Tplus
                
                # Compute HTC
                res = HTC.run_htc(mat, r1, r2, Tmin, Tmax, nT, steps, eq_steps, runs, step_clust=1,
                                  norm=True, Tdiv_log=False, display=False)
        
                # Store dimension
                np.savetxt(folder+data.full_names[idx], np.stack(res))